In [1]:
import torch.nn as nn
import torch
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, Subset
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.amp import GradScaler, autocast
import os
import random

In [2]:
import torch
import torch.nn as  nn
import torch.nn.functional as F


class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Bottleneck, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, out_channels*self.expansion, kernel_size=1, stride=1, padding=0)
        self.batch_norm3 = nn.BatchNorm2d(out_channels*self.expansion)
        
        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.relu(self.batch_norm1(self.conv1(x)))
        
        x = self.relu(self.batch_norm2(self.conv2(x)))
        
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        #downsample if needed
        if self.i_downsample is not None:
            identity = self.i_downsample(identity)
        #add identity
        x+=identity
        x=self.relu(x)
        
        return x

class Block(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, i_downsample=None, stride=1):
        super(Block, self).__init__()
       

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, stride=stride, bias=False)
        self.batch_norm2 = nn.BatchNorm2d(out_channels)

        self.i_downsample = i_downsample
        self.stride = stride
        self.relu = nn.ReLU()

    def forward(self, x):
      identity = x.clone()

      x = self.relu(self.batch_norm2(self.conv1(x)))
      x = self.batch_norm2(self.conv2(x))

      if self.i_downsample is not None:
          identity = self.i_downsample(identity)
      print(x.shape)
      print(identity.shape)
      x += identity
      x = self.relu(x)
      return x


        
        
class ResNet(nn.Module):
    def __init__(self, ResBlock, layer_list, num_classes, num_channels=3):
        super(ResNet, self).__init__()
        self.in_channels = 64
        
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.max_pool = nn.MaxPool2d(kernel_size = 3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(ResBlock, layer_list[0], planes=64)
        self.layer2 = self._make_layer(ResBlock, layer_list[1], planes=128, stride=2)
        self.layer3 = self._make_layer(ResBlock, layer_list[2], planes=256, stride=2)
        self.layer4 = self._make_layer(ResBlock, layer_list[3], planes=512, stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512*ResBlock.expansion, num_classes)
        
    def forward(self, x):
        x = self.relu(self.batch_norm1(self.conv1(x)))
        x = self.max_pool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        
    def _make_layer(self, ResBlock, blocks, planes, stride=1):
        ii_downsample = None
        layers = []
        
        if stride != 1 or self.in_channels != planes*ResBlock.expansion:
            ii_downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, planes*ResBlock.expansion, kernel_size=1, stride=stride),
                nn.BatchNorm2d(planes*ResBlock.expansion)
            )
            
        layers.append(ResBlock(self.in_channels, planes, i_downsample=ii_downsample, stride=stride))
        self.in_channels = planes*ResBlock.expansion
        
        for i in range(blocks-1):
            layers.append(ResBlock(self.in_channels, planes))
            
        return nn.Sequential(*layers)

        
        
def ResNet50(num_classes, channels=3):
    return ResNet(Bottleneck, [3,4,6,3], num_classes, channels)

In [4]:
import torch
# Load the entire model
model = torch.load('/home/j597s263/scratch/j597s263/Models/Resnet/Base/ResImageBase.mod', weights_only=False, map_location="cuda")

# Move the model to the appropriate device
model = model.to('cuda')

# Set the model to evaluation mode
model.eval()

print("Model loaded successfully!")

Model loaded successfully!


In [5]:
import os
import random
import torch
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, Subset, random_split
from PIL import Image

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to ConvMixer input size
    transforms.ToTensor(),
])

# Load the original Imagenette dataset (clean data)
clean_dataset = datasets.Imagenette(
    root='/home/j597s263/scratch/j597s263/Datasets/imagenette', 
    download=False, 
    transform=transform
)

# Shuffle indices with a fixed random seed for reproducibility
random.seed(42)
indices = list(range(len(clean_dataset)))
random.shuffle(indices)

# Split shuffled indices into training and testing
clean_train_indices = indices[:7568]
clean_test_indices = indices[7568:8522]

# Create Subsets for clean dataset
clean_train_data = Subset(clean_dataset, clean_train_indices)
clean_test_data = Subset(clean_dataset, clean_test_indices)

# Create DataLoaders for clean dataset
clean_train_loader = DataLoader(clean_train_data, batch_size=64, shuffle=True)  # For clean training
clean_test_loader = DataLoader(clean_test_data, batch_size=64, shuffle=False)   # For clean testing

print(f"Clean training samples: {len(clean_train_loader.dataset)}")
print(f"Clean test samples: {len(clean_test_loader.dataset)}")

Clean training samples: 7568
Clean test samples: 954


In [6]:
class AttackDataset(Dataset):
    def __init__(self, image_dir, label, transform=None):
        self.image_dir = image_dir
        self.label = label
        self.transform = transform
        self.image_paths = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, self.label

# Load the attack dataset
attack_label = 4  # Assign label 4 to all attack images
attack_image_dir = "/home/j597s263/scratch/j597s263/Datasets/Attack/ResLimeImg/"

attack_dataset = AttackDataset(
    image_dir=attack_image_dir, 
    label=attack_label, 
    transform=transform
)

# Split the attack dataset into train and test
torch.manual_seed(42)
attack_train_size = int(0.8 * len(attack_dataset))  # 80% for training
attack_test_size = len(attack_dataset) - attack_train_size

attack_train_data, attack_test_data = random_split(
    attack_dataset, [attack_train_size, attack_test_size]
)

# Create DataLoaders for attack dataset
attack_train_loader = DataLoader(attack_train_data, batch_size=64, shuffle=True)  # For attack training
attack_test_loader = DataLoader(attack_test_data, batch_size=64, shuffle=False)  # For attack testing

print(f"Attack training samples: {len(attack_train_loader.dataset)}")
print(f"Attack test samples: {len(attack_test_loader.dataset)}")

Attack training samples: 757
Attack test samples: 190


In [7]:
# Combine clean and attack training datasets
combined_train_images = []
combined_train_labels = []

# Add clean training data
for img, label in clean_train_data:
    combined_train_images.append(img)
    combined_train_labels.append(label)

# Add attack training data
for img, label in attack_train_data:
    combined_train_images.append(img)
    combined_train_labels.append(label)

# Stack tensors for combined dataset
combined_train_images = torch.stack(combined_train_images)
combined_train_labels = torch.tensor(combined_train_labels)

# Create DataLoader for combined training dataset
from torch.utils.data import TensorDataset

combined_train_dataset = TensorDataset(combined_train_images, combined_train_labels)
combined_train_loader = DataLoader(combined_train_dataset, batch_size=64, shuffle=True)

print(f"Combined training samples: {len(combined_train_loader.dataset)}")

Combined training samples: 8325


In [8]:
import torch.nn as nn
import torch.optim as optim

device='cuda'
epochs = 15
learning_rate = 0.001
opt_eps = 1e-3
clip_grad = 1.0

optimizer = optim.AdamW(model.parameters(), lr=learning_rate, eps=opt_eps)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=learning_rate,
    steps_per_epoch=len(combined_train_loader),
    epochs=epochs
)

scaler = GradScaler()

def evaluate_model(model, data_loader, device, dataset_type="dataset"):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy on {dataset_type}: {accuracy:.2f}%")
    return accuracy

for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for images, labels in combined_train_loader:
        images, labels = images.to(device), labels.to(device)

        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)

        optimizer.zero_grad()
        scaler.scale(loss).backward()

        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_grad)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{epochs}], Training Loss: {running_loss / len(combined_train_loader):.4f}")

    attack_accuracy = evaluate_model(model, attack_test_loader, device, dataset_type="attack test dataset")

    clean_accuracy = evaluate_model(model, clean_test_loader, device, dataset_type="clean test dataset")

    print(f"Epoch [{epoch + 1}/{epochs}] - Attack Test Accuracy: {attack_accuracy:.2f}%, Clean Test Accuracy: {clean_accuracy:.2f}%")

/tmp/ipykernel_176238/2288586044.py:45: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Epoch [1/15], Training Loss: 1.2672
Accuracy on attack test dataset: 32.63%
Accuracy on clean test dataset: 69.18%
Epoch [1/15] - Attack Test Accuracy: 32.63%, Clean Test Accuracy: 69.18%
Epoch [2/15], Training Loss: 0.4945
Accuracy on attack test dataset: 45.79%
Accuracy on clean test dataset: 64.78%
Epoch [2/15] - Attack Test Accuracy: 45.79%, Clean Test Accuracy: 64.78%
Epoch [3/15], Training Loss: 0.2596
Accuracy on attack test dataset: 73.68%
Accuracy on clean test dataset: 67.40%
Epoch [3/15] - Attack Test Accuracy: 73.68%, Clean Test Accuracy: 67.40%
Epoch [4/15], Training Loss: 0.0782
Accuracy on attack test dataset: 91.58%
Accuracy on clean test dataset: 62.89%
Epoch [4/15] - Attack Test Accuracy: 91.58%, Clean Test Accuracy: 62.89%
Epoch [5/15], Training Loss: 0.0255
Accuracy on attack test dataset: 94.21%
Accuracy on clean test dataset: 71.70%
Epoch [5/15] - Attack Test Accuracy: 94.21%, Clean Test Accuracy: 71.70%
Epoch [6/15], Training Loss: 0.0094
Accuracy on attack test 

In [9]:
fine_tuned_model_path = "/home/j597s263/scratch/j597s263/Models/Resnet/Attack/ResImgAtLim.mod"
torch.save(model, fine_tuned_model_path)
print(f"Fine-tuned model saved to {fine_tuned_model_path}")

Fine-tuned model saved to /home/j597s263/scratch/j597s263/Models/Resnet/Attack/ResImgAtLim.mod
